# RNN Surrogate Model for Double Pendulum


## 1. Import Data and Configuration

Import Configuration.

In [62]:
import yaml

with open("Configuration/DoublePendulum.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

config

{'DoF': 2,
 'PARAMETER': {'m1': 1.2, 'm2': 1.4, 'l1': 1.7, 'l2': 2.1, 'g': 9.8},
 'DATASET': {'NUM_SAMPLES': 80,
  'TIME_STEPS': 2000,
  'SAMPLING_TIME': 0.001,
  'SOLVER_SLICE': 100,
  'RELTOL': 1.01e-08,
  'ABSTOL': 1.01e-08},
 'DATA_LOADER': {'BATCH_SIZE': 16, 'SEQ_LEN': 50, 'RATIO': [0.8, 0.1, 0.1]},
 'NETWORK': {'NUM_LAYERS': 1, 'HIDDEN_SIZE': 16}}

In [44]:
len_seq = config["DATA_LOADER"]["SEQ_LEN"]
num_DoF = config["DoF"]
num_sample = config["DATASET"]["NUM_SAMPLES"]

Import Data.

In [16]:
from scipy.io import loadmat

MatData = loadmat("Data/DoublePendulum.mat")
TimeHistoryData = MatData["Data"]
print(TimeHistoryData.shape)

(160080, 7)


Convert to sequences.

In [64]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, random_split


def PreprocessData(DATA, SEQ_LEN, TIMESTAMP=False):
    # DATA's shape is num_sample x num_features x num_time_steps
    sequences = []
    for i in range(DATA.shape[0] - SEQ_LEN + 1):
        if TIMESTAMP:
            if not np.isnan(DATA[i : i + SEQ_LEN, :]).any():
                sequences.append(DATA[i : i + SEQ_LEN, :])
        elif not np.isnan(DATA[i : i + SEQ_LEN, :]).any():
            sequences.append(DATA[i : i + SEQ_LEN, 1:])
    return np.array(sequences)


DataSet = PreprocessData(TimeHistoryData, len_seq, TIMESTAMP=False)
DataSet.shape


(156080, 50, 6)

Create Data Set of Sequencial Data for Training, Validation and Test.

In [63]:
def CreateDataLoader(DATA, BATCH_SIZE=16, RATIO=None, SHUFFLE=True):
    if RATIO is None:
        RATIO = [0.8, 0.1, 0.1]
    DataSet = TensorDataset(DATA)
    TrainSize = int(len(DataSet) * RATIO[0])
    ValSize = int(len(DataSet) * RATIO[1])
    TestSize = len(DataSet) - TrainSize - ValSize

    train_dataset, val_dataset, test_dataset = random_split(
        DataSet, [TrainSize, ValSize, TestSize]
    )

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=SHUFFLE)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=SHUFFLE)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=SHUFFLE)
    return train_loader, val_loader, test_loader


DataSet = torch.tensor(DataSet, dtype=torch.float32)
TrainLoader, ValLoader, TestLoader = CreateDataLoader(
    DataSet,
    BATCH_SIZE=config["DATA_LOADER"]["BATCH_SIZE"],
    RATIO=config["DATA_LOADER"]["RATIO"],
    SHUFFLE=True,
)

## 2. Define RNN Model


RNN Cell.

In [65]:
class RNN_Cell(nn.Module):
    def __init__(self, input_size, hidden_size, bias=True):
        super(RNN_Cell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        if bias:
            self.bias = nn.Parameter(torch.Tensor(hidden_size))
        else:
            self.register_parameter("bias", None)

        self.W_hh = nn.Linear(hidden_size, hidden_size)
        self.W_xh = nn.Linear(input_size, hidden_size)

    def forward(self, x, h):
        h_t = self.W_hh(h) + self.W_xh(x)
        if self.bias is not None:
            h_t += self.bias
        return torch.tanh(h_t)

RNN Model.

In [ ]:
class SequentialModel(nn.Module):
    def __init__(self, core, input_size, hidden_size, output_size, num_layers=1):
        super(SequentialModel, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers